# Amazon Review Pre-processing

Most of the work is done in classes outside of this notebook, but I'll outline what I did for pre-processing in this notebook. Running pre-processing in the notebook took a lot longer so running it via command line seems to be quicker so I can iterate

Code is located in util folder:
* preprocess_amazon.py - python program that calls TextProcessor with parameters set to handle the amazon review file
* TextProcessor.py - processor class that uses various utilities to pre-process the file
* text_util.py - has functions to do text processing
* file_util.py - functiont to handle files (ie, covert tsv to csv)
* df_util.py - utility to handle pandas DataFrames


Unit Tests:
* TestTextUtil.py - tests text_util.py

To Be Implemented:
* unit tests for file_util.py
* unit tests for pd_util.py


## General ML Workflow

Since most of the work is actually done outside of Jupyter notebook, there is a series of python programs in *tools* directory that were created to pre-process and train our models.

The following diagram outlines the process:

<img src="../images/ml-workflow.png" alt="General ML Workflow" />


In [1]:
# import sibling utilities
import sys
sys.path.append('..')

import util.file_util as fu
from util.file_util import convert_tsv_to_csv
import pandas as pd
import util.text_util as tu
import util.file_samplers as fs
import logging

logging.basicConfig(level=logging.INFO)

# only need to run this one time
CONVERT_FILE=False

# full 9mil Wireless reviews
ORIG_FILE_WIRELESS="../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00.tsv"

# List of potential CSV files to feed pre-processor
DATA_FILE_50K = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-50k.csv"
DATA_FILE_100K = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k.csv"
DATA_FILE_200K = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-200k.csv"
DATA_FILE_500K = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-500k.csv"
DATA_FILE_1m = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-1m.csv"
DATA_FILE_2m = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-2m.csv"
DATA_FILE_4m = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-4m.csv"




# Convert Amazon files to CSV format and Downsample Our Files

Before we pre-process, I had to convert tsv to CSV because Pandas was not reading the columns correctly and was putting multiple rows into a column resulting in headline columns that had over 30k words

Original Amazon file had 9mil reviews. I added ability to down-sampling our files in this step since I wanted to reduce the size of files that we use for our prototypes. Currently, the sampling is pretty dumb. It just grabs every nth line in the file and put it in the final csv file. Will probably rewrite this so it's based on random.rand later or use pandas sample function


In [2]:
??fu.convert_tsv_to_csv

Signature: fu.convert_tsv_to_csv(infile: str, outfile: str, sampler=None)
Source:   
def convert_tsv_to_csv(infile: str, outfile: str, sampler = None):
    """
    Read a tsv file line by line then covert it into a readable csv file
    :param infile: input TSV file
    :param outfile: output CSV file
    :param sampler - Sampler to reduce the output file size
    :return:
    """

    print(f"Converting {infile} to {outfile} with sampling {sampler}")

    if os.path.isfile(infile):

        counter = 0
        with open(infile, "r+") as old_f, open(outfile, "w") as new_f:
            writer = csv.writer(new_f)
            for line in old_f:
                # never sample 0 because that's the header column
                if sampler is None or sampler.collect(counter):
                    data = line.strip('\n').split('\t')
                    writer.writerow(data)
                counter += 1
        print(f"Finished converting {infile} to {outfile} with sampling {sampler}")
    else:

In [3]:
??fs.SimpleSampler

Init signature: fs.SimpleSampler(sample_rate: int, has_header: bool = True)
Source:        
class SimpleSampler(Sampler):
    """
    Simple sampling means that we are going to keep every X entry. X is specified by the sample_rate in the constructor

    Every X samples will be saved to the file
    """

    def __init__(self, sample_rate:int, has_header:bool = True):
        super().__init__(has_header)
        self.sample_rate = sample_rate

    def collect(self, index:int) -> bool:
        """
        returns True if we want to keep this file
        :param index:
        :return:
        """
        # if file has header and we are at the first line, then keep
        if self.sample_rate == 0:
            return True
        if self.has_header and index == 0:
            return True
        if index % self.sample_rate == 0:
            return True
        return False
File:           ~/Dropbox/0_springboard/capstone/util/file_samplers.py
Type:           type
Subclasses:     


#### Convert files to CSV only needs to be done once. To rerun, set CONVERT_FILE to True

In [2]:
if CONVERT_FILE:
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_4m, fs.SimpleSampler(2))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_2m, fs.SimpleSampler(4))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_1m, fs.SimpleSampler(9))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_500K, fs.SimpleSampler(18))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_200K, fs.SimpleSampler(45))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_100K, fs.SimpleSampler(90))
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_50K, fs.SimpleSampler(180))

Converting ../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00.tsv to ../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-4m.csv with sampling <util.file_samplers.SimpleSampler object at 0x7fe4107064d0>
Finished converting ../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00.tsv to ../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-4m.csv with sampling <util.file_samplers.SimpleSampler object at 0x7fe4107064d0>


# Preprocssing Amazon Review File

## To do pre-processing. Run the following command:
```
cd ../tools
python amazon_review_preprocessor.py -l INFO -o ../dataset/amazon_reviews ../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k.csv
```

Output file is located at:
```
../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k-preprocessed.csv
```

## Pre-processing Steps

Pre-processing entails the following steps (in order):
* make everything lowercase
* remove newlines
* remove amazon tags - amazon embeds these [[VIDDEO:dsfljlsjf]] and [[ASIN:sdfjlsjdfl]] tags that need to be removed
* remove html tags - line breaks, etc are represented in reviews as HTML tags
* remove accent characters
* expand contractions - expands contractions like he's but needs to be done before special charaters because we want to expand don't into do not for our text processing
* remove special characters - anything that is not alphanumeric or spaces
* remove stop words - see text_util.py for stop words that I removed from nltk stop words because I think they will be important for sentiment analysis
* lemmatize words - lemmatize using wordnet to use only base words


Finally, the pre-processor will remove any entries with 0-length reviews. The output file after pre-processing has 99567 entries instead of 100k because of this.

## Class Diagarm

<img src="../images/amazon_review_preprocessor-class-diagram.png" alt="Amazon Review Preprocessor" style="width: 600px;"/>

# The remaining sections shows code snippets of what each of the pre-processing steps does

## make lower case

In [5]:
??tu.make_lowercase

Signature: tu.make_lowercase(text: str) -> str
Source:   
def make_lowercase(text: str) -> str:
    """
    Make text lower case
    :param text: original text
    :return: lower case string
    """
    return text.lower()
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## remove new lines

In [6]:
??tu.remove_newlines

Signature: tu.remove_newlines(text: str) -> str
Source:   
def remove_newlines(text: str) -> str:
    """
    remove newlines from text. will stirp both unix and windows
    newline characters

    :return: string without newlines
    """
    # logger.debug(f'pre-stripped: [{text}]')
    newtext = text.replace('\n', '').replace('\r', '')
    return ' '.join(newtext.split())
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## remove html tags

In [7]:
??tu.remove_html_tags

Signature: tu.remove_html_tags(text: str) -> str
Source:   
def remove_html_tags(text: str) -> str:
    """
    Remove HTML taxs from text usig regex
    :param text: original text
    :return: stripped text
    """
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## remove accented characters

In [8]:
??tu.remove_accented_chars

Signature: tu.remove_accented_chars(text: str) -> str
Source:   
def remove_accented_chars(text: str) -> str:
    """
    Remove accent characters and convert to UTF-8
    :param text: original text
    :return: stripped text
    """
    text = unicodedata.normalize('NFKD', text).encode(
        'ascii', 'ignore').decode('utf-8', 'ignore')
    return text
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## expand contractions

In [9]:
??tu.expand_contractions

Signature:
tu.expand_contractions(
    text: str,
    contraction_mapping={"ain't": 'is not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', 'gonna': 'going to', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "i'd": 'i would', "i'd've": 'i would have', "i'll": 'i will', "i'll've": 'i will have', "i'm": 'i am', 'im': 'I am', "i've": 'i have', "isn't": 'is not', "it'd": 'it would', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "might

## remove special characters

In [10]:
??tu.remove_special_chars

Signature: tu.remove_special_chars(text: str) -> str
Source:   
def remove_special_chars(text: str) -> str:
    """
    Remove anything that is not alphanumeric or spaces
    :param text:
    :return:
    """
    text = re.sub('[^a-zA-Z0-9\s]', ' ', text, flags=re.I | re.A)
    text = remove_newlines(text)
    return ' '.join(text.split())
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## remove alphanumeric characters

In [11]:
??tu.remove_alphanumeric_words

Signature: tu.remove_alphanumeric_words(x)
Source:   
def remove_alphanumeric_words(x):
    """
    In amazon reviews especially for wireless categories you have some alpha numeric word
    which represent model numbers, we want to remove this

    Will also remove words that are basically only numbers
    :param x:
    :return:
    """
    # remove mixed words
    x = re.sub(r'\s*([a-z]+[\d]+[\w\d]*|[\d]+[a-z]+[\d\w]*|[\d]{2,})', '', x, flags=re.IGNORECASE)
    # remove numbers
    # x = re.sub(r'\s(\d+)', '', x)
    return ' '.join(x.split())
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## remove stop words

I removed the following stop words from NLTK because in a review context, if I say 'like' or 'not like' makes a difference in a review context

The following words were removed from NLTK's default stop word list:

[    'no',
    'not',
    'do',
    'don',
    "don't",
    'does',
    'did',
    'does',
    'doesn',
    "doesn't",
    'should',
    'very',
    'will']

In [12]:
??tu.remove_stop_words

Signature: tu.remove_stop_words(text: str) -> str
Source:   
def remove_stop_words(text: str) -> str:
    """
    remove stop words from string
    :param text: original text
    :return: string without stop words
    """
    if text is not None and len(text) > 0:
        try:
            tokens = wpt.tokenize(text)
            filtered_tokens = [word for word in tokens if word not in stop_words]
            return ' '.join(filtered_tokens)
        except:
            traceback.print_exc()
            logger.error(f"got error trying to tokenize this string [{text}]")
            sys.exit(1)
    return text
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


## lemmatize words

lemmatization changes the word to the base word - ie, car, cars, car's to car

In [13]:
??tu.lemmatize_text

Signature: tu.lemmatize_text(text: str) -> str
Source:   
def lemmatize_text(text: str) -> str:
    """
    lemmatize work
    :param text:
    :return:
    """

    logger.debug(f'lemmatizing text: {text}')
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output
File:      ~/Dropbox/0_springboard/capstone/util/text_util.py
Type:      function


# Inspect results in our Pre-processed file

In [14]:

PREPROCESSED_CSV = "../dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k-preprocessed.csv"


## Reading the output file back in to look at some data

* there should be no empty columns or rows

In [15]:
review_df = pd.read_csv(PREPROCESSED_CSV, parse_dates=["review_date"])
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99567 entries, 0 to 99566
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   star_rating      99567 non-null  int64         
 1   helpful_votes    99567 non-null  int64         
 2   total_votes      99567 non-null  int64         
 3   review_headline  99567 non-null  object        
 4   review_body      99567 non-null  object        
 5   review_date      99567 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.6+ MB


### Let's sample the data

In [16]:
# let's sample the dataframe so we can look at some of the data
pd.set_option('display.max_colwidth', 150)
review_df.review_body.sample(10).values

array(['case really smart looking little slippery hand car charger doe not fit opening take using charger',
       'various tough phone nextel lastest samsun meet ti amely di amise eventually thats got warranty used day le test abuse warranty warranty awesome week turn around destruction replacement call quality pretty good speaker get bit muffled dunk make call otherwise ringer loud vibrate worthless battery life 6 7 day texting day constant flashlight useage do not net can not comment use phone camera work great software come worthless plug use usb storage device transfer photo also no built photo editing option hand durable phone ever 3 year will trash get new one dy warranty will buy another will never buy regular phone did mention water proof ever get call shower take easiest way clean grit grease use hand nail brush scrub clean week',
       'grandaughter 2 year old glide tricycle not expected truly looked forward seeng glide normal height sits down leg bend much uncomfortable po